<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/%D0%91%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8%20Python%20%D0%B4%D0%BB%D1%8F%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0/%D0%94%D0%97_%D0%9A%D0%BE%D0%BD%D0%B4%D1%80%D0%B0%D1%82%D1%8C%D0%B5%D0%B2_NUMPY-48%20%D0%BA%20%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'

In [27]:
res = requests.get('https://habr.com/ru/articles/')
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.text)
soup

In [ ]:
news = soup.find_all('article', class_='tm-articles-list__item')
print(len(news))
print(news[0])

In [35]:
for article in news:
    title_element = article.find('h2', class_='tm-megapost-snippet__title')  # поиск элемента с заголовком
    if title_element is None:
        title_element = article.find('a', class_='tm-title__link')  # если не найден, ищем элемент с ссылкой
    if title_element is not None:
        title = title_element.text.strip()  # получаем текст заголовка
        print(title)
        relative_link = title_element.get('href')  # получаем относительную ссылку
        full_link = urllib.parse.urljoin(base_url, relative_link)  # создаем полную ссылку
        print(full_link)
        date_element = article.find('span', class_='tm-article-datetime-published').find('time')
        if date_element is None:
            date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
        if date_element is not None:
            date = date_element.get('title')
            print(date)
            print()

Кратчайший путь с одним источником во взвешенных графах, Алгоритм Дейкстры и Python
https://habr.com/ru/companies/otus/articles/771016/
2023-10-31, 14:28

Технология «Дятел»: новаторское решение проблемы галлюцинаций ИИ
https://habr.com/ru/companies/first/articles/771000/
2023-10-31, 14:15

Как мы создаём защищённые автоматизированные рабочие места
https://habr.com/ru/companies/inferit/articles/770998/
2023-10-31, 14:15

9 советов по выбору шлагбаума
https://habr.com/ru/companies/perco/articles/770994/
2023-10-31, 14:00

Международная гонка в области квантовых технологий и интернета набирает обороты
https://habr.com/ru/companies/itglobalcom/articles/770956/
2023-10-31, 13:00

Арифмометр Чебышёва: первая отечественная счетная машина с непрерывным движением
https://habr.com/ru/companies/rshb/articles/770968/
2023-10-31, 12:53

Дайджест событий для эйчаров и рекрутеров в IT в ноябре
https://habr.com/ru/companies/habr_career/articles/770972/
2023-10-31, 12:46

Олег Бартунов о премии Highlo

In [ ]:
def get_habr_posts(query):
    habr_blog = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title_element = article.find('a', 'tm-title__link')
            title = title_element.text
            relative_link = title_element.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            date_element = article.find('span', class_='tm-article-datetime-published').find('time')
            date = date_element.get('title')
            row = {'date': date, 'title': title, 'link': full_link}
            habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['анализ данных'])
res

In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

def get_habr_posts(query):
    base_url = 'https://habr.com'
    habr_blog = pd.DataFrame()
    for q in query:
        URL = f'{base_url}/ru/search/'
        params = {'q': q}
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title_element = article.find('h2', class_='tm-megapost-snippet__title')  # поиск элемента с заголовком
            if title_element is None:
                title_element = article.find('a', class_='tm-title__link')  # если не найден, ищем элемент с ссылкой
            if title_element is not None:
                title = title_element.text.strip()  # получаем текст заголовка
                print(title)
                relative_link = title_element.get('href')  # получаем относительную ссылку
                full_link = urllib.parse.urljoin(base_url, relative_link)  # создаем полную ссылку
                print(full_link)
                date_element = article.find('span', class_='tm-article-datetime-published').find('time')
                if date_element is None:
                    date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
                if date_element is not None:
                    date = date_element.get('title')
                    print(date)
                row = {'date': date, 'title': title, 'link': full_link}
                habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
print(res)


Как устроен GIL (Global Interpreter Lock) в Python: влияние на многозадачность и производительность
https://habr.com/ru/companies/otus/articles/769448/
2023-10-24, 13:48
Python-разработчик: из новичка в профессионалы. Как начать свой путь в освоении языка?
https://habr.com/ru/companies/productstar/articles/769134/
2023-10-22, 19:07
Go vs Python. Виталий Левченко
https://habr.com/ru/articles/488644/
2020-02-17, 10:34
Что выбрать новичку: Python или Java?
https://habr.com/ru/companies/productstar/articles/769680/
2023-10-25, 13:34
Python Lives in Excel
https://habr.com/ru/articles/770006/
2023-10-26, 08:42
У нас в Excel поселился замечательный Python
https://habr.com/ru/articles/769732/
2023-10-25, 08:26
Искусственный интеллект на Python с использованием TensorFlow и Keras
https://habr.com/ru/articles/770554/
2023-10-29, 12:40
Python декораторы на максималках. Универсальный рецепт по написанию и аннотированию от мала до велика
https://habr.com/ru/articles/750312/
2023-07-25, 20:02
Решени

AttributeError: ignored

In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

def get_habr_posts(query):
    base_url = 'https://habr.com'
    habr_blog = pd.DataFrame()

    for q in query:
        URL = f'{base_url}/ru/search/'
        params = {'q': q}
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        articles = soup.find_all('article', class_='tm-articles-list__item')

        for article in articles:
            title_element = article.find('h2', class_='tm-megapost-snippet__title')  # поиск элемента с заголовком

            if title_element is None:
                title_element = article.find('a', class_='tm-title__link')  # если не найден, ищем элемент с ссылкой
                title = title_element.text.strip()  # получаем текст заголовка
                relative_link = title_element.get('href')  # получаем относительную ссылку
                full_link = urllib.parse.urljoin(base_url, relative_link)  # создаем полную ссылку
                date_element = article.find('span', class_='tm-article-datetime-published').find('time')
                date = date_element.get('title')

            if title is not None:
                title = title_element.text.strip()  # получаем текст заголовка
                relative_link = article.find('a', 'full_link')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date = article.find('time', class_='tm-megapost-snippet__datetime-published')

        row = {'date': date, 'title': title, 'link': full_link}
        habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])

    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
res

,date,title,link
0,None,Три уровня погружения в Python. Запись докладо...,https://habr.com
1,None,Обзор наиболее интересных материалов по анализ...,https://habr.com


In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

def get_habr_posts(query):
    base_url = 'https://habr.com'
    posts = []

    for q in query:
        URL = f'{base_url}/ru/search/'
        params = {'q': q}
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        articles = soup.find_all('article', class_='tm-articles-list__item')

        for article in articles:
            title = article.find('h2', class_='tm-megapost-snippet__title')
            if title is None:
                title = article.find('a', class_='tm-title__link')

            if title is not None:
                title = title.text.strip()
                relative_link = title.get('href')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date_element = article.find('span', class_='correct-class-name').find('time')
                date = date_element.get('title') if date_element else None
                posts.append({'date': date, 'title': title, 'link': full_link})

    return pd.DataFrame(posts).reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
res

AttributeError: ignored

Код выполняет следующие действия:

1. Создание функции `get_habr_posts`, которая принимает в качестве параметра список `query`. В этой функции будет выполнен поиск статей на сайте habr.com по каждому ключевому слову из списка `query`.

2. Создание пустого DataFrame с именем `habr_blog`, в котором будут храниться данные о статьях.

3. В цикле `for q in query` происходит перебор каждого ключевого слова в списке `query`. Для каждого ключевого слова инициализируется GET-запрос к URL-адресу "https://habr.com/ru/search/" со значением параметра `q` в качестве поискового запроса.

4. Затем выполняется парсинг HTML-страницы с помощью BeautifulSoup, чтобы найти все статьи на странице. При помощи метода `find_all` ищутся все элементы `<article>` с классом `"tm-articles-list__item"`.

5. Внутри вложенного цикла `for article in articles` происходит обработка каждого элемента `<article>`, чтобы извлечь информацию о дате публикации, названии статьи и ссылке на нее.

6. Извлеченные данные сохраняются в словаре `row` с ключами `'date'`, `'title'` и `'link'`.

7. С помощью конкатенации DataFrame (`pd.concat([habr_blog, pd.DataFrame([row])])`) данные из словаря `row` добавляются как новая строка в DataFrame `habr_blog`.

8. В конце функция возвращает DataFrame `habr_blog`, содержащий информацию о статьях.

9. После вызова функции `get_habr_posts(['python', 'data science'])` результат сохраняется в переменной `res`.

10. Для удаления дубликатов в DataFrame `res` применяется метод `drop_duplicates()`. Этот метод удаляет полностью дублирующие строки из DataFrame.

11. Затем результаты выводятся на экран путем вызова `res`.

## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```
